In [25]:
import cv2
import numpy as np
import os, csv

In [26]:
def scale_range(input, min, max):
    input += -(np.min(input))
    input /= np.max(input) / (max - min)
    input += min
    return input

In [27]:
def avg(arr):
    return np.average(arr)

In [40]:
def read_files():
    file_names = list()
    GLF, SGR, VI_GREEN, RGR = [],[],[],[]
    chloro_results = list()
    nitro_results = list()
    leaf_areas = list()
    for dirpath, dirnames, filenames in os.walk("."):
        for filenames in [f for f in filenames if f.endswith(('.JPG','.jpg','.png'))]:
            #print(filenames)
            img=cv2.imread(filenames)
            #print('Image size {}'.format(img.size))
            
            [r,c,l] = img.shape
            #print(r)
            #print(c)
            #print(l)
                
            R = img[:,:,2]
            G = img[:,:,1]
            B = img[:,:,0]
            #print(R,G,B)
            #print(filenames)
            
            # Chrolophyll content calculation
            for i in np.arange(1,r).reshape(-1):
                for j in np.arange(1,c).reshape(-1):
                    if B[i,j] > 90:
                        R[i,j]=0
                        G[i,j]=0
                        B[i,j]=0

            R1=np.mean(R)
            G1=np.mean(G)
            B1=np.mean(B)

            chloro = G1 - (R1 / 2) - (B1 / 2)
            print('Red Channel mean is:',round(R1))
            print('Green Channel mean is:',round(G1))
            print('Blue Channel mean is:',round(B1))
            print('Chrolophyll content is:',round(chloro))


            # Nitrogen content calculation
            R_avg = np.mean(R) / 255
            G_avg = np.mean(G) / 255
            B_avg = np.mean(B) / 255
            maxi = np.max((R_avg,G_avg,B_avg))
            mini = np.min((R_avg,G_avg,B_avg))

            if maxi == R_avg:
                H = np.dot(((G_avg - B_avg) / (maxi -  mini)), 60)
            if maxi == G_avg:
                H=np.dot((((B_avg - R_avg) / (maxi - mini)) + 2),60)
            if maxi == B_avg:
                H=np.dot((((R_avg - G_avg) / (maxi - mini)) + 4),60)

            S=(maxi - mini) / maxi
            B=maxi.copy()
            nitro=((H - 60) / 60 + (1 - S) + (1 - B)) / 3
            print('Nitrogen Content is:',nitro)

            # for leaf area perform transformation & calculation as in PYM research paper

            h, w = img[:,:,0].shape # get original image shape
            print(h)
            print(w)
            pym = np.zeros((h, w),np.int) # blank b/w image for storing pym image
            red = np.zeros((h, w),np.int) # blank array for red
            blue = np.zeros((h, w),np.int) # blank array for blue

            # Specific channels
            red = (img[:,:,2]).astype('float') # reading red channel from original image (NIR)
            blue = (img[:,:,0]).astype('float') # reading blue channel from original image (blue)

            # PYM calculation
            max_sc = np.amax(red - blue/2)
            pym = ((red - blue/2)*255/max_sc).astype('uint8') # computing new channel

            pym[red - blue/2 < 0] = 0 # setting limit

            '''
            # False color image
            False_color_image = np.zeros((r, c,3),np.uint8) # make a blank RGB image 
            False_color_image[:,:,1] = pym
            False_color_image[:,:,2] = 255 - pym

            #f_image = "FALSE_COLOR_" + filename 
            #f_dest = "FALSE_COLOR/" + f_image 
            #cv2.imwrite(f_dest, False_color_image) 
            
            #print('pym: {}'.format(pym))
            '''
            
            # Image analysis

            ret,thresh1 = cv2.threshold(pym, 0, 255, cv2.THRESH_OTSU) # OTSU's thresholding  #pym
            kernel_open = np.ones((6,6),np.uint8) # large kernel
            kernel_mid = np.ones((4,4),np.uint8) # medium kernel
            kernel_close = np.ones((2,2),np.uint8) # small kernel
            kernel_veryclose = np.ones((1,1),np.uint8) # tiny petit

            erosion = cv2.erode(thresh1, kernel_veryclose,iterations = 1) # edge erosion
            opening = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel_open) # removing noise around the plant
            closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel_mid) # removing noise inside the plant
            #contours, hierarchy = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) # finding plant contours
            aa, contours, hierarchy = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) # with an older version of opencv
            '''
            area = []
            for contr in contours:
                ar = cv2.contourArea(contr)
                area.append(ar)
            print('check {}'.format(area))

            sorted_area = sorted(area, key=int, reverse = True)
            leaf_area = sorted_area[0] # largest area, plant area by definition
            leaf_area_index = area.index(leaf_area) # finding area index 
            cnt = contours[leaf_area_index] # plant contours, with holes included
            cv2.drawContours(closing, [cnt], 0,(255,0,0),-1) # drawing contours with holes included
            '''
            
            if include_holes==1: # Counting all pixels inside the largest area
                areas = [] # list
                for contour in contours:
                    ar = cv2.contourArea(contour)
                    areas.append(ar)
                print('what is this {}'.format(areas))

                sorted_area = sorted(areas, key=int, reverse = True)
                leaf_area = sorted_area[0] # largest area, plant area by definition
                leaf_area_index = areas.index(leaf_area) # finding area index 
                cnt = contours[leaf_area_index] # plant contours, with holes included
                cv2.drawContours(closing, [cnt], 0,(255,0,0),-1) # drawing contours with holes included
            #print('LeafArea: {}'.format(leaf_area))
            
            if include_holes==0: # Counting all pixels detected
                cv2.drawContours(closing, contours, -1, (255, 255, 255), -1) # drawing contours without including holes
                leaf_area = (closing > 127).sum() # couting plants pixels
            print('LeafArea: {}'.format(leaf_area))
            
            
            # Green leaf index
            top = 2*G-R-B
            down = 2*G+R+B
            down[down == 0] = 1
            temp = top/down
            print(temp)
            print(avg(temp))
            scaled = scale_range(temp, 0, 255)
            print(scaled)
            
            # summed green
            sg = sum(G)
            print(sg)
            print(avg(sg))
            
            # VI_GREEN
            top1 = G-R
            down1 = G+R
            down1[down1 == 0] = 1
            temp1 = temp1 = top1/down1
            print(temp1)
            print(avg(temp1))
            scaled1 = scale_range(temp1, 0, 255)
            print(scaled1)
            
            # RED-TO-GREEN
            G[G==0] = 255*100
            temp2 = R/G
            print(temp2)
            print(avg(temp2))
            scaled2 = scale_range(temp2, 0, 255)
            print(scaled2)
            
            file_names.append(filenames[:-4])
            chloro_results.append(chloro)
            nitro_results.append(nitro)
            leaf_areas.append(leaf_area)
            GLF.append(avg(temp))
            SGR.append(avg(sg))
            VI_GREEN.append(avg(temp1))
            RGR.append(avg(temp2))
            
        return file_names, chloro_results, nitro_results, leaf_areas, GLF, SGR, VI_GREEN, RGR

In [43]:
def writefile():
    with open('my_data.csv','w', newline='') as myfile:
        wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
        wr.writerow(['File_ID','chloro_content','nitrogen_content','leaf_area_pixel','GLF','SGR','VI_GREEN','RGR']) # 
        wr.writerows(zip(f,c,n,la,gl,sg_r,vi,rg_r)) #

In [44]:
if __name__ =="__main__":
    include_holes = 1 # highest area (lettuce in field)
    include_holes = 0 # counting white pixels (other cases)
    f,c,n,la,gl,sg_r,vi,rg_r = read_files() #
    print('Filename: {} Chloro_result: {} nitro_result: {} LeafArea: {} GreenLeafIndex: {}  SummedGreenReflectance: {}  Vigreen: {}  RedGreenRatio: {}' .format(f,c,n,la,gl,sg_r,vi,rg_r))
    writefile()

Red Channel mean is: 26.0
Green Channel mean is: 30.0
Blue Channel mean is: 23.0
Chrolophyll content is: 6.0
Nitrogen Content is: 0.7457246104835541
1961
2201
LeafArea: 1696323
[[ 0.93767239  0.96642368  0.98214032 ...  0.83784279  0.77083747
   0.61555319]
 [ 0.99804729 -1.         -1.         ... -1.         -1.
  -1.        ]
 [ 1.11300185 -1.         -1.         ... -1.         -1.
  -1.        ]
 ...
 [ 0.2843694   0.28160092  0.29592808 ...  0.09677816  0.06527852
   0.08961831]
 [ 0.28302322  0.2843694   0.29592808 ...  0.08961831  0.06527852
   0.10352934]
 [ 0.31888665  0.31968102  0.32004633 ...  0.07391456  0.07391456
   0.07391456]]
1.6966535881570073
[[0.32924706 0.33413245 0.336803   ... 0.31228413 0.30089866 0.27451294]
 [0.33950589 0.         0.         ... 0.         0.         0.        ]
 [0.35903884 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.21823857 0.21776815 0.22020261 ... 0.18636328 0.1810109  0.18514669]
 [0.21800983 0.21823857 0.22020

In [3]:
def green_leaf_index(r,g,b):
    top = 2*g-r-b
    down = 2*g+r+b
    down[down == 0] = 1
    temp = top/down
    scaled = scale_range(temp, 0, 255)
    return temp, scaled

In [4]:
def summed_green_reflectance(g):
    return sum(g)

In [5]:
def vi_green(g,r):
    top = g-r
    down = g+r
    down[down == 0] = 1
    temp = temp = top/down
    scaled = scale_range(temp, 0, 255)
    return temp, scaled

In [6]:
def red_to_green_ratio(g,r):
    g[g==0] = 255*100
    scaled = scale_range(r/g, 0, 255)
    return r/g, np.array(scaled, dtype = np.uint8 )

In [7]:
def carotenoid_reflectance_index(g):
    range_500

In [8]:
def split_rgb(img):
    red = img[:,:,2]
    green = img[:,:,1]
    blue = img[:,:,0]
    return red, green, blue

In [9]:
def read_image(name):
    image = cv2.imread(name)
    return image

In [10]:
def avg(arr):
    return np.average(arr)

In [11]:
def display(img):
    cv2.imshow("", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
# calculating vegetation indexes

r,g,b = split_rgb(read_image("1.png"))

In [13]:
glf, scaled_glf = green_leaf_index(r,g,b)
print(glf)
print(avg(glf))
#print(scaled_glf)

[[1.11904762e+01 1.80769231e+01 1.96666667e+01 ... 1.04545455e+01
  4.60000000e+00 1.38554217e+00]
 [4.70000000e+01 1.38235294e+01 7.37500000e+00 ... 3.10810811e+00
  2.67441860e+00 1.76923077e+00]
 [9.83333333e-01 1.96666667e+01 5.90000000e+00 ... 2.16981132e+00
  2.80487805e+00 2.94871795e+00]
 ...
 [1.75862069e+00 1.86131387e+00 1.40845070e-02 ... 2.11864407e+00
  2.45098039e+00 2.19298246e+00]
 [1.80851064e+00 1.75862069e+00 1.40845070e-02 ... 2.19298246e+00
  2.45098039e+00 2.04918033e+00]
 [2.25563910e-02 2.12765957e-02 2.06896552e-02 ... 2.35849057e+00
  2.35849057e+00 2.35849057e+00]]
3.5102587226755


In [14]:
VIgreen, VIgreen_scaled = vi_green(g,r)
print(avg(VIgreen))

3.607743083805034


In [15]:
r_to_g, r_to_g_scaled = red_to_green_ratio(r, g)
print(avg(r_to_g))

1.1838327778758573


In [16]:
sgr = summed_green_reflectance(g)
print(avg(sgr))

128.22716946842345
